In [70]:
#Importação das bibliotecas
import pandas as pd
import datetime as dt
import plotly.graph_objs as go
import plotly.express as px

In [9]:
#leitura dos atributos comuns às duas bases#

with open('attributes.txt', 'r') as atributos:
    atributos.read()

dfa = pd.read_csv('attributes.txt')
dfa.rename(columns={'# Attributes for both netflix_titles.csv and amazon_prime_titles.csv datasets:': 'Atributos em comum para os datasets netflix_titles.csv e amazon_prime_titles.csv'}, inplace=True)
dfa

,Atributos em comum para os datasets netflix_titles.csv e amazon_prime_titles.csv
0,1 show_id - Unique ID for every Movie / Tv Show
1,2 type - Identifier - A Movie or TV Show
2,3 title - Title of the Movie / Tv Show
3,4 director - Director of the Movie
4,5 cast - Actors involved in the movie / show
5,6 country - Country where the movie / show was...
6,7 data_added - Date it was added on Netflix or...
7,8 release_year - Actual Release year of the mo...
8,9 rating - TV Rating of the movie / show
9,10 duration - Total Duration - in minutes or n...


In [12]:
#leitura das bases amazon e netflix#

dfamazon = pd.read_csv('amazon_prime_titles.csv', encoding='latin-1')
dfamazon.insert(2, 'Empresa','Amazon')
dfnetflix = pd.read_csv('netflix_titles.csv', encoding='latin-1')
dfnetflix.insert(2, 'Empresa','Netflix')

#conferencia do tipo de dados das fontes

'''dfamazon.dtypes
dfnetflix.dtypes'''

#concatenando as bases, definindo chave para identificação, renomeando os campos e estruturação#

geral = pd.concat([dfamazon, dfnetflix], ignore_index= True)

geral['id'] = geral['Empresa'] + geral['show_id']

geral.rename(columns={'show_id':'id_original','type': 'Tipo', 'title': 'Titulo', 'director': 'Diretor', 'cast': 'Elenco', 'country': 'País', 'date_added': 'Data_Adicao', 'release_year': 'Lançamento', 'rating': 'Classificacao', 'duration': 'Duracao', 'listed_in': 'Genero', 'description': 'Descricao'}, inplace= True)

geral['Data_Adicao'] = geral['Data_Adicao'].astype('datetime64')

Análise Exploratória 

Tamanho do Dataset

In [ ]:
geral.head
geral.tail
geral.shape
display(geral)

Descobrindo o período da análise.

In [96]:
inicio = pd.to_datetime(geral['Data_Adicao']).dt.date
print('O período de início é',min(inicio))

fim = pd.to_datetime(geral['Data_Adicao']).dt.date
print('O período final é',max(inicio))

O período de início é 2008-01-01
O período final é 2021-10-10


C:\Users\T-Gamer\AppData\Local\Temp\ipykernel_7476\218204634.py:2: FutureWarning:

Comparison of NaT with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable.

C:\Users\T-Gamer\AppData\Local\Temp\ipykernel_7476\218204634.py:5: FutureWarning:

Comparison of NaT with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable.



Valores nulos e tipos de dados

In [ ]:
geral.info()

Verificando o tipo de dados

In [ ]:
geral.dtypes

Verificando a existência de valores nulos

In [ ]:
geral.isnull().sum()

Analisando informações estatísticas

In [ ]:
geral.describe()

Filtrar dos dados

1 - Top 10 atores/atrizes considerando todos os dados

In [75]:
#ajuste do dataframe para explodir o elenco que continha mais de um por célula do dataframe e retirada dos outlaiers do elenco.
geral_elenco_explode = geral.assign(Elenco = geral['Elenco'].str.split(',')).explode('Elenco')
indices_a_remover = geral_elenco_explode.loc[geral_elenco_explode['Elenco'] == '1'].index
geral_elenco_filtro = geral_elenco_explode.drop(index=indices_a_remover)

In [76]:
#criação da tabela com o ranking dos artistas
geral_elenco_filtro = geral_elenco_filtro['Elenco'].value_counts().head(10).rename_axis('Artista').to_frame('Quantidade')
geral_elenco_filtro.reset_index(inplace=True, drop=False)
geral_elenco_filtro['Rank'] = geral_elenco_filtro.index
geral_elenco_filtro['Rank'] = geral_elenco_filtro['Rank'] + 1
geral_elenco_filtro = geral_elenco_filtro[['Artista', 'Quantidade', 'Rank']]
tabela1_elenco = go.Figure(data=[go.Table(
    header=dict(values=list(geral_elenco_filtro.columns)),
    cells=dict(values=[geral_elenco_filtro.Artista, geral_elenco_filtro.Quantidade, geral_elenco_filtro.Rank]))
])
tabela1_elenco.show()

In [103]:
fig = px.bar(geral_elenco_filtro,
            x = 'Artista', # coluna para os dados do eixo x
            y = 'Quantidade', # coluna para os dados do eixo y
            text='Quantidade',
            color='Artista',                      
            #hover_name = 'Artista'
            )
fig.update_layout(xaxis = dict(linecolor='rgba(0,0,0,1)', # adicionando linha em y = 0
                                tickmode = 'array', # alterando o modo dos ticks
                                tickvals = geral_elenco_filtro['Quantidade'], # setando o valor do tick de x
                                ticktext = geral_elenco_filtro['Artista']), # setando o valor do tick de x
                 yaxis = dict(linecolor='rgba(0,0,0,1)', # adicionando linha em x = 0
                              tickformat=False), # removendo a formatação no eixo y
                title_text = ' Artistas com Maiores Participações em Séries e Filmes entre Janeiro de 2008 e Outubro de 2021' # adicionando titulo ao gráfico
                ,title_x = 0.5)
fig.show()